In [28]:
from new_main import instantiate_agents, instantiate_auction
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from collections import defaultdict
import json
import pickle
from copy import deepcopy
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
class Publisher:
    def __init__(self, rng, name, num_auctions):
        self.rng = rng
        self.name = name
        # Read from memory publisher embedding
        self.embedding = pickle.load(open("publisher_embedding/data/sites/" + name + ".pkl", "rb"))
        # Number of auctions to simulate according to past data
        self.num_auctions = num_auctions
        # Metrics of revenue
        self.revenue = .0

    def generate_user_context(self):
        # Definisci l'intensità del rumore
        noise_strength = 0.01
        # Genera il rumore gaussiano
        noise = np.random.normal(0, noise_strength, self.embedding.shape)
        # Aggiungi il rumore agli embedding originali
        noisy_embeddings = self.embedding + noise
        return noisy_embeddings

In [4]:
def parse_config(path):
    with open(path) as f:
        config = json.load(f)

    # Set up Random Number Generator
    rng = np.random.default_rng(config['random_seed'])
    np.random.seed(config['random_seed'])

    # Number of runs
    num_runs = config['num_runs'] if 'num_runs' in config.keys() else 1

    # Max. number of slots in every auction round
    # Multi-slot is currently not fully supported.
    max_slots = 1

    # Expand agent-config if there are multiple copies
    agent_configs = []
    num_agents = 0
    for agent_config in config['agents']:
        if 'num_copies' in agent_config.keys():
            for i in range(1, agent_config['num_copies'] + 1):
                agent_config_copy = deepcopy(agent_config)
                agent_config_copy['name'] += f' {num_agents + 1}'
                agent_configs.append(agent_config_copy)
                num_agents += 1
        else:
            agent_configs.append(agent_config)
            num_agents += 1

    # Read ads embeddings
    ads_embeddings = pickle.load(open('publisher_embedding/data/nuova_descr_annunci_embed.pkl', 'rb'))
    # Pick embeddings from a pre-defined set
    agents2items = {
        agent_config['name']: ads_embeddings[agent_config['product_name']]
        for agent_config in agent_configs
    }

    agents2item_values = {
        #agent_config['name']: rng.lognormal(0.1, 0.2, agent_config['num_items'])
        agent_config['name']: np.ones(agent_config['num_items'])
        for agent_config in agent_configs
    }

    # Publishers configs
    publisher_configs = config['publishers']

    return (rng, config, agent_configs, agents2items, agents2item_values, publisher_configs,
            num_runs, max_slots)

In [10]:
def instantiate_publishers(rng, publisher_configs):
    return [
        Publisher(
            rng=rng,
            name=publisher_config['name'],
            num_auctions=publisher_config['num_auctions'],
        )
        for publisher_config in publisher_configs
    ]

In [11]:
# Parse configuration file
(rng, config, agent_configs, agents2items, agents2item_values, publisher_configs, num_runs, max_slots) = parse_config('../new_config/FP_Oracle_auto_DR_others_Truth.json')

In [12]:
agents = instantiate_agents(rng, agent_configs, agents2item_values, agents2items)
auction, num_iter, rounds_per_iter, output_dir = instantiate_auction(rng, config, agents2items, agents2item_values, agents, max_slots)
publishers = instantiate_publishers(rng, publisher_configs)

In [18]:
def add_noise(user_embedding, noise_strength):
    noise = np.random.normal(0, noise_strength, user_embedding.shape)
    return user_embedding + noise

In [22]:
def compute_user_embed_matrix(publisher, noise_strength):
    user_embed_matrix = np.zeros((publisher.num_auctions, publisher.embedding.size))
    for i in range(publisher.num_auctions):
        user_embed_matrix[i] = add_noise(publisher.embedding, noise_strength)
    return user_embed_matrix

In [24]:
def compute_all_users_matrix(publishers, noise_strength):
    all_users_matrix = {}
    for publisher in publishers:
        all_users_matrix[publisher.name] = compute_user_embed_matrix(publisher, noise_strength)
    return all_users_matrix

In [20]:
noise_strength = 0.01

In [25]:
all_users_matrix = compute_all_users_matrix(publishers, noise_strength)

In [32]:
def agent_pub_user_similarity(agents2items, publisher, all_users_matrix):
    user_pub_sim = {}
    curr_pub_matrix = all_users_matrix[publisher.name]
    for agent, ad_embedding in agents2items.items():
        sim = cosine_similarity(ad_embedding.reshape(1, -1), curr_pub_matrix)
        user_pub_sim[agent] = sim
    return user_pub_sim

In [34]:
def compute_matrix_user_ad(agents2items, publishers, all_users_matrix):
    matrix_user_ad = {}
    for publisher in publishers:
        matrix_user_ad[publisher.name] = agent_pub_user_similarity(agents2items, publisher, all_users_matrix)
    return matrix_user_ad

In [35]:
matrix_user_ad = compute_matrix_user_ad(agents2items, publishers, all_users_matrix)

In [49]:
matrix_user_ad["corriere.it"]['Auto da corsa 2'][:, 1]

array([0.19729596])

In [36]:
def compute_all_users_agent_similarity(publishers, noise_strength, agents2items):
    all_users_matrix = compute_all_users_matrix(publishers, noise_strength)
    return compute_matrix_user_ad(agents2items, publishers, all_users_matrix)

In [37]:
noise_strength = 0.01
all_users_agent_similarity = compute_all_users_agent_similarity(publishers, noise_strength, agents2items)

In [38]:
pickle.dump(all_users_agent_similarity, open("all_users_agent_similarity.pkl", "wb"))

In [40]:
mask_pub_agent = {}
for publisher in publishers:
    mask_pub_agent[publisher.name] = np.zeros(publisher.num_auctions)

In [ ]:
# Pick at random a publisher and take the first index of the relative array different from 1. Repeat until all the arrays are filled with 1
def fill_mask_pub_agent(mask_pub_agent, publishers):
    while not all(np.all(mask == 1) for mask in mask_pub_agent.values()):
        publisher = np.random.choice(publishers)
        mask = mask_pub_agent[publisher.name]
        idx = np.where(mask == 0)[0][0]
        
        mask[idx] = 1

In [ ]:
def run_repeated_auctions():
    agents = instantiate_agents(rng, agent_configs, agents2item_values, agents2items)
    auction, num_iter, rounds_per_iter, output_dir = instantiate_auction(rng, config, agents2items, agents2item_values, agents, max_slots)
    publishers = instantiate_publishers(rng, publisher_configs)
    
    agent2net_utility = defaultdict(list)
    agent2gross_utility = defaultdict(list)
    
    for i in range(num_iter):
        print(f'==== ITERATION {i} ====')
        # Round-robin over publishers
        pub_auctions = {publisher.name: publisher.num_auctions for publisher in publishers}
        total_auctions = sum(pub_auctions.values())

        with tqdm(total=total_auctions) as pbar:
            while sum(pub_auctions.values()) > 0:
                for publisher in publishers:
                    if pub_auctions[publisher.name] > 0:
                        # Generate a user from the current publisher
                        curr_user_context = publisher.generate_user_context()
                        # Simulate the auction
                        auction.simulate_opportunity(curr_user_context, publisher.name)
                        pub_auctions[publisher.name] -= 1
                        pbar.update(1)

        for agent_id, agent in enumerate(auction.agents):
            print(f'Agent {agent.name} has spent {agent.get_agent_spent()}, '
                  f'winning {agent.get_agent_opp_won()} opportunities (win ratio: {agent.get_agent_won_ratio()})')
            print(f'Mean value paid for a won bid: {agent.get_mean_won_bid()}')

            agent.update(iteration=i)

            agent.clear_utility()
            agent.clear_logs()

        auction.clear_revenue()
        
    return agent2net_utility, agent2gross_utility

In [ ]:
import os

In [ ]:
# Parse configuration file
(rng, config, agent_configs, agents2items, agents2item_values, publisher_configs, num_runs, max_slots) = parse_config('../new_config/FP_Oracle_auto_DR_others_Truth.json')

In [ ]:
agents = instantiate_agents(rng, agent_configs, agents2item_values, agents2items)
auction, num_iter, rounds_per_iter, output_dir = instantiate_auction(rng, config, agents2items, agents2item_values, agents, max_slots)
publishers = instantiate_publishers(rng, publisher_configs)

In [ ]:
def get_random_publishers(num_publishers, num_auctions):
    sites_dir = "src/publisher_embedding/data/sites/"
    sites = [site.replace(".pkl", "") for site in os.listdir(sites_dir)]
    chosen_site_list = np.random.choice(sites, num_publishers, replace=False)
    return [
        {
            "name": site,
            "num_auctions": num_auctions
        }
        for site in chosen_site_list
    ]

In [ ]:
noise_strength = 0.01
all_user_context, all_users_agent_similarity = compute_all_users_agent_similarity(publishers, noise_strength, agents2items)